In [2]:
import os
import pandas as pd
from deepface import DeepFace
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

# === Paths ===
image_folder = "/Users/anielamac/Desktop/thesis/PL_emotion_game_MANUAL/dataset CK/ck_images"
labels_csv = "/Users/anielamac/Desktop/thesis/PL_emotion_game_MANUAL/dataset CK/ck_converted_for_game.csv"
output_csv = "/Users/anielamac/Desktop/thesis/PL_emotion_game_MANUAL/dataset CK/ck_emotion_difficulty.csv"

# === Load ground truth labels ===
df_labels = pd.read_csv(labels_csv)

# === Filter out Neutral and Contempt ===
df_labels = df_labels[~df_labels["emotion"].str.lower().isin(["neutral", "contempt"])].reset_index(drop=True)

# === Run DeepFace analysis ===
emotion_results = []

print("Analyzing images with DeepFace...")

for idx, row in tqdm(df_labels.iterrows(), total=len(df_labels)):
    filename = row["filename"]
    correct_label = row["emotion"].lower()
    img_path = os.path.join(image_folder, filename)

    try:
        result = DeepFace.analyze(img_path=img_path, actions=["emotion"], enforce_detection=False)[0]
        emotions = result["emotion"]
        correct_score = emotions.get(correct_label, 0)
        raw_difficulty = 1.0 - (correct_score / 100)

        emotion_results.append({
            "filename": filename,
            "folder_label": correct_label,
            **emotions,
            "raw_difficulty": raw_difficulty
        })

    except Exception as e:
        print(f"Skipped {filename} - {e}")

# === Create dataframe ===
df = pd.DataFrame(emotion_results)

# === Normalize difficulty per emotion class ===
df["difficulty"] = 0.0  # initialize

for emo in df["folder_label"].unique():
    mask = df["folder_label"] == emo
    scaler = MinMaxScaler()
    df.loc[mask, "difficulty"] = scaler.fit_transform(df.loc[mask, ["raw_difficulty"]])

# === Final cleanup ===
df.drop(columns=["raw_difficulty"], inplace=True)
df["shown"] = False
df = df.sort_values("difficulty").reset_index(drop=True)

# === Save CSV ===
df.to_csv(output_csv, index=False)
print(f"\n✅ Done! Saved to: {output_csv}")


Analyzing images with DeepFace...


100%|██████████| 309/309 [02:48<00:00,  1.83it/s]



✅ Done! Saved to: /Users/anielamac/Desktop/thesis/PL_emotion_game_MANUAL/dataset CK/ck_emotion_difficulty.csv
